Example of maximum-likelihood fit with iminuit version 2.
pdf is a mixture of Gaussian (signal) and exponential (background),
truncated in [xMin,xMax].
G. Cowan / RHUL Physics / December 2021

We first will start with importing all the necessary packages. The exercise is based on python implementation of Minuit package. We will use it to perform an unbinned MLE fit to the generated mass-spectrum with one peak. 

In [3]:
import numpy as np
import scipy.stats as stats
from scipy.stats import truncexpon #truncated exponential in [a,b]
from scipy.stats import truncnorm  #truncated normal distribution in [a,b]
from scipy.stats import chi2
import iminuit
from iminuit import Minuit
import matplotlib.pyplot as plt
from matplotlib import container
plt.rcParams["font.size"] = 14
print("iminuit version:", iminuit.__version__)  # need 2.x

iminuit version: 2.31.1


Now, here we will upload the dataset generated for you beforehand. 

In [4]:
import pandas as pd

df = pd.read_csv("../datasets/dataset.csv", header=None)
xData = df.values  

The model we will use is rather simple. The signal is modelled with a single Gaussian, while the background is modelled with exponential. The total PDF is defined as following :
$$f(m) = \theta \cdot G(m; \mu, \sigma) + (1 - \theta)\cdot Exp(m; \xi)$$
where $\theta$ is signal fraction and $\xi$ is exponential power. 

In [ ]:
def f(x, par):
    theta   = par[0]
    mu      = par[1]
    sigma   = par[2]
    xi      = par[3]
    fs = stats.truncnorm.pdf(x, a=(xMin-mu)/sigma, b=(xMax-mu)/sigma, loc=mu, scale=sigma)
    fb = stats.truncexpon.pdf(x, b=(xMax-xMin)/xi, loc=xMin, scale=xi)
    return theta*fs + (1-theta)*fb

We will now have to implement the nLL ourselves. Note, that Minuit expects the nLL python function to depend ONLY on the parameters. 

In [ ]:
def negLogL(par):
    pdf = f(xData, par)
    return -np.sum(np.log(pdf))

Task 1: Configure the iminuit fit. For this we need to setup the 2 POI $\theta$ and $\xi$ and 2 nuisance parameters $\mu$ and $\sigma$. 
1. We configure initial values
2. We configure initial step sizes for the parameter minimization.
3. We configure the limits, if necessary.

   

In [ ]:
# Initialize Minuit and set up fit:
parin   = np.array([theta, mu, sigma, xi]) # initial values (here = true values)
parname = ['theta', 'mu', 'sigma', 'xi']


#parstep = np.array([0.1, 1., 1., 1.])      # initial setp sizes 
#parfix  = [False, True, True, False]       # change these to fix/free parameters
#parlim  = [(0.,1), (None, None), (0., None), (0., None)]    # set limits

parstep = np.array([1000., 1000., 1000., 1000.])      # initial setp sizes 
parfix  = [False, False, False, False]       # change these to fix/free parameters
parlim  = [(None, None), (None, None), (None, None), (None, None)]    # set limits
m = Minuit(negLogL, parin, name=parname)
m.errors = parstep
m.fixed = parfix
m.limits = parlim
m.errordef = 0.1                          # errors from lnL = lnLmax - x

Now let's dot he fit and get the covariance.

In [ ]:
# Do the fit, get errors, extract results
m.migrad()                                        # minimize -logL
MLE = m.values                                    # max-likelihood estimates
sigmaMLE = m.errors                               # standard deviations
cov = m.covariance                                # covariance matrix
rho = m.covariance.correlation()                  # correlation coeffs.

Check the parameters values and standard deviation. 

In [ ]:
print(r"par index, name, estimate, standard deviation:")
for i in range(m.npar):
    if not m.fixed[i]:
        print("{:4d}".format(i), "{:<10s}".format(m.parameters[i]), " = ",
         "{:.6f}".format(MLE[i]), " +/- ", "{:.6f}".format(sigmaMLE[i]))

In [ ]:
print()
print(r"free par indices, covariance, correlation coeff.:")
for i in range(m.npar):
    if not(m.fixed[i]):
        for j in range(m.npar):
            if not(m.fixed[j]):
                print(i, j, "{:.6f}".format(cov[i,j]), "{:.6f}".format(rho[i,j]))

In [ ]:
# Plot fitted pdf
yMin = 0.
yMax = f(0., MLE)*1.1
fig = plt.figure(figsize=(8,6))
xCurve = np.linspace(xMin, xMax, 100)
yCurve = f(xCurve, MLE)
plt.plot(xCurve, yCurve, color='dodgerblue')

In [ ]:
# Plot data as tick marks
tick_height = 0.05*(yMax - yMin)
xvals = [xData, xData]
yvals = [np.zeros_like(xData), tick_height * np.ones_like(xData)]
plt.plot(xvals, yvals, color='black', linewidth=1)
plt.xlabel(r'$x$')
plt.ylabel(r'$f(x; \theta)$')
plt.figtext(0.6, 0.8, r'$\hat{\theta} = $' + f'{MLE[0]:.4f}' +
            r'$\pm$' + f'{sigmaMLE[0]:.4f}')
plt.figtext(0.6, 0.72, r'$\hat{\xi} = $' + f'{MLE[3]:.4f}' +
            r'$\pm$' + f'{sigmaMLE[3]:.4f}')
plt.xlim(xMin, xMax)
plt.ylim(yMin, yMax)
plt.show()

In [ ]:
# Make scan of lnL (for theta, if free)
if not(m.fixed['theta']):
    plt.figure()
    m.draw_mnprofile('theta')
    plt.show()

In [ ]:
# Make a contour plot of lnL = lnLmax - 1/2 (here for theta and xi).
# The tangents to this contour give the standard deviations.
CL = stats.chi2.cdf(1.,2)            #  Q_alpha = 1, npar = 2
print('CL = ', CL)
if not(m.fixed['theta'] | m.fixed['xi']):
    fig, ax = plt.subplots(1,1)
    con = m.mncontour('theta', 'xi', cl=CL, size=200)
    con = np.vstack([con, con[0]])         # close contour
    plt.plot(MLE[0], MLE[3], marker='o', linestyle='None', color='black', label=r'$(\hat{\theta}, \hat{\xi})$')
    plt.plot(con[:,0], con[:,1], color='black', linewidth=1)
    plt.xlabel(r'$\theta$', labelpad=5)
    plt.ylabel(r'$\xi$', labelpad=5)
    handles, labels = ax.get_legend_handles_labels()
    plt.legend(handles, labels, loc='upper right', fontsize=14, frameon=False)
    plt.figtext(0.4, 0.93, r'$\ln L = \ln L_{\rm max} - 1/2$')
    plt.show()

In [ ]:
# Confidence region from lnL = lnLmax - Q/2 (here for theta and xi)
# where Q is the chi2 quantile of CL = 1-alpha = 0.683 and 0.95 for 2 dof.
if not(m.fixed['theta'] | m.fixed['xi']):
    fig, ax = plt.subplots(1,1)
    m.draw_mncontour('theta', 'xi', cl=[0.683, 0.95], size=200);
    plt.plot(MLE[0], MLE[3], marker='o', linestyle='None', color='black', label=r'$(\hat{\theta}, \hat{\xi})$')
    plt.xlabel(r'$\theta$', labelpad=10)
    plt.ylabel(r'$\xi$', labelpad=10)
    plt.subplots_adjust(left=0.2, right=0.9, top=0.9, bottom=0.2)
    handles, labels = ax.get_legend_handles_labels()
    plt.legend(handles, labels, loc='upper right', fontsize=14, frameon=False)
    plt.figtext(0.3, 0.93, r'$\ln L = \ln L_{\rm max} - \frac{1}{2} F^{-1}_{\chi^2}(1-\alpha;n)$')
    plt.show()

Task 2: Now checkout dataset1-5. Repeat the fit for all of them and plot the change of the $\sigma(\theta)$ versus sample size. Which relation do you observe?

In [6]:
#Here is your solution
import ROOT
import zfit

/srv/conda/envs/notebook/lib/python3.10/site-packages/zfit/__init__.py:60: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warning, set ZFIT_DISABLE_TF_WARNINGS=1.
  warnings.warn(
2025-05-08 20:56:34.387876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746737794.401367     579 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746737794.405356     579 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746737794.417354     579 computation_placer.cc:177] computation placer already registered